In [ ]:
#Imports

import os
import librosa
import numpy as np
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from pydub import AudioSegment

In [ ]:
# Load data

# Initialize empty list to store features and corresponding labels
mfcc_features = []
labels = []
# Manual genre mapping
genre_mapping = {'blues':0, 'classical':1, 'country':2, 'disco':3, 'hiphop':4, 'jazz':5, 'metal':6, 'pop':7, 'reggae':8, 'rock':9}
# Initialize path of the dataset, sample ratio, segement duration and overlap
dataset_path = "/content/drive/MyDrive/Data"
segment_duration = 5
overlap = 0.5
SR = 22050
# SR = 44100
n_fft=2048
hop_length=512

# Iterate through each genre folder
for genre in os.listdir(dataset_path):
    genre_path = os.path.join(dataset_path, genre)

    # Iterate through each audio file in the genre folder
    for filename in os.listdir(genre_path):
       music_path = os.path.join(genre_path, filename)
       try:
           # Load the audio file
           y, sr = librosa.load(music_path, sr=SR)

           # Calculate the number of samples per segment
           segment_samples = int(segment_duration * sr)

           # Calculate the number of samples to overlap
           overlap_samples = int(overlap * segment_samples)

           # Extract MFCC features for each segment
           for i in range(0, len(y) - segment_samples + 1, segment_samples - overlap_samples):
                segment = y[i:i + segment_samples]

                # Extract MFCC features
                mfcc = librosa.feature.mfcc(y=segment, sr=sr, n_mfcc=13,n_fft=n_fft, hop_length=hop_length)
                mfcc = mfcc.T

                # Append the MFCC features and corresponding label
                mfcc_features.append(mfcc)
                labels.append(genre_mapping[genre])

       except Exception as e:
            print(f"Error processing {music_path}: {e}")

In [ ]:
# Function for pitch shifting
def pitch_shift(mfcc, sr, n_steps=2):
    return librosa.effects.pitch_shift(y=mfcc, sr=sr, n_steps=n_steps)

# Data augmentation parameters
pitch_shift_steps = 2
# Data augmentation after collecting features and labels
augmented_mfcc_features = []
augmented_labels = []

for mfcc, label in zip(mfcc_features, labels):
    augmented_mfcc = pitch_shift(mfcc.flatten(), SR, n_steps=pitch_shift_steps).reshape(mfcc.T.shape).T
    augmented_mfcc_features.append(augmented_mfcc)
    augmented_labels.append(label)

In [ ]:
# Extend the original features and labels with augmented ones
mfcc_features.extend(augmented_mfcc_features)
labels.extend(augmented_labels)

In [ ]:
# Convert the labels to numpy array
labels = np.array(labels)

In [ ]:
# Reshape and normalize the mfcc features
mfcc1_features = [mfcc.reshape((mfcc.shape[0], mfcc.shape[1], 1)) for mfcc in mfcc_features]
mfcc1_features = np.array(mfcc1_features)
mfcc1_features = (mfcc1_features - np.mean(mfcc1_features)) / np.std(mfcc1_features)

In [ ]:

# Specify the folder path
folder_path = '/content/drive/MyDrive/Models'

# Ensure the folder exists, create it if necessary
os.makedirs(folder_path, exist_ok=True)

# Save the NumPy array to the folder
np.save(os.path.join(folder_path, 'mfcc1_features.npy'), mfcc1_features)
np.save(os.path.join(folder_path, 'labels.npy'), labels)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(mfcc1_features, labels, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
# One-hot encode labels
y_train_onehot = to_categorical(y_train,num_classes=10)
y_val_onehot = to_categorical(y_val,num_classes=10)
y_test_onehot = to_categorical(y_test,num_classes=10)

In [ ]:
# Calculate mean and standard deviation along the axis of features
mean_values_normalized = [float(np.mean(mfcc1_features))]
std_deviations_normalized = [float(np.std(mfcc1_features))]

# Print the mean and standard deviation for each feature
for i, (mean, std) in enumerate(zip(mean_values_normalized, std_deviations_normalized), 3):
    print(f"Feature {i}: Mean = {mean:.3f}, Standard Deviation = {std:.3f}")

In [ ]:
input_shape = (X_train.shape[1], X_train.shape[2], 1)
model = Sequential()

# 1st conv layer
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
model.add(BatchNormalization())

# 2nd conv layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
model.add(BatchNormalization())

# 3rd conv layer
model.add(Conv2D(256, (2, 2), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
model.add(BatchNormalization())

# flatten output and feed it into dense layer
model.add(Flatten())
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))

# output layer
num_classes = len(np.unique(y_train_onehot, axis=0))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# train model
history = model.fit(X_train, y_train_onehot, validation_data=(X_val, y_val_onehot), batch_size=32, epochs=50, callbacks=[early_stopping])

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test_onehot)
print(f'Test Accuracy: {accuracy:.2f}')

In [ ]:
import matplotlib.pyplot as plt

# Plot training history
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
